In [ ]:
  !nvidia-smi

In [ ]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())

In [ ]:
import pandas as pd
import numpy as np
import copy
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.utils.vis_utils import plot_model
from keras import optimizers
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
import torch
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv("PM-Day-data.txt",names=["area","cycle_id","hour","position_name","station_code","aqi","quality" ,"pm2_5","pm2_5_24h" ,"pm10" ,"pm10_24h" , "so2","so2_24h" , "no2","no2_24h", "co","co_24h", "o3","o3_8h","o3_8h_24h","o3_24h","primary_pollutant","drop"],header = None,parse_dates=True, index_col='cycle_id',encoding='gb18030')

In [ ]:
data_chang = data[data['area']=='常州']
data_chang = data_chang.drop('drop',axis=1)
data_c = data_chang[data_chang['position_name']=='市监测站']

In [ ]:
data_c = data_c.drop(['area','hour','position_name','station_code','aqi','quality','primary_pollutant'],axis=1)
data_c.replace('--',np.nan,inplace=True)
data_c = data_c.iloc[11:,:]
data_c = data_c.fillna(method='ffill')
data_c['pm2_5_24h'] = data_c['pm2_5_24h'].astype(float)
data_c['pm10_24h'] = data_c['pm10_24h'].astype(float)
data_c['so2_24h'] = data_c['so2_24h'].astype(float)
data_c['no2_24h'] = data_c['no2_24h'].astype(float)
data_c['co_24h'] = data_c['co_24h'].astype(float)
data_c['o3_8h'] = data_c['o3_8h'].astype(float)
data_c['o3_8h_24h'] = data_c['o3_8h_24h'].astype(float)
data_c['o3_24h'] = data_c['o3_24h'].astype(float)
print(data_c.info())

# import data

In [ ]:
dataset = pd.read_csv("data_multi_cleaned_10_final.csv",encoding='gb18030')

In [ ]:
# dataset = dataset.iloc[:,0:7]
print(dataset.head())

In [ ]:
var_origin=dataset.values
print(var_origin.shape)

In [ ]:
0.8704,0.6881,0.7141,0.7418,0.6655,0.7921

In [ ]:
location_weight = [1,0.9640,0.9625,0.9518,0.9240,0.8905,0.8863,0.8730,0.8719,0.8575,0.8527]
location_weight = [val for val in location_weight for i in range(7)]
domain_weight = [1,0.8702,0.6893,0.7150,0.7416,0.6655,0.7929]
domain_weight = domain_weight*11

In [ ]:
location_weight = [1,0.9640,0.9625,0.9518,0.9240,0.8905,0.8863,0.8730,0.8719,0.8575,0.8527]
location_weight = [val for val in location_weight for i in range(7)]
domain_weight = [1,0.8316,0.7083,0.7059,0.7500,0.6060,0.7344]
domain_weight = domain_weight*11

In [ ]:
print(location_weight)
print(domain_weight)

In [ ]:
weight = [a*b for a,b in zip(data_weight,domain_weight)]
print(weight)

In [ ]:
weight_d = pd.read_csv('weight.csv', encoding='gb18030')
data_weight = np.array(weight_d['weight'])
location = np.array(weight_d['location'])

In [ ]:
data_wei = data_weight[1:7]
print(data_wei)

In [ ]:
data_sery = pd.DataFrame(np.ones(3250))
del_list = list()
weight_list = list()
data_w = list()
domain = list()
loc = list()
for i in range(77):
    if weight[i] < 0.6:
        del_list.append(i)
    else:
        weight_list.append(weight[i])
        data_w.append(data_weight[i])
        domain.append(domain_weight[i])
        loc.append(location[i])
origin = np.delete(var_origin,del_list,1)

In [ ]:
print(len(weight_list))
print(len(data_w))
print(len(domain))
print(origin.shape)

In [ ]:
print(loc)
print(weight_list)

Normalize data

In [ ]:
origin = var_origin

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
scale = scaler.fit_transform(var_origin)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
scale = scaler.fit_transform(origin)

In [ ]:
# scaled_data = pd.DataFrame(scaled)
# scaled_data.plot(subplots=True, figsize=(20,24))

In [ ]:
for i in range(20):
    scale[:,i] = scale[:,i]*weight_list[i]

In [ ]:
for i in range(77):
    scale[:,i] = scale[:,i]*data_w[i]

In [ ]:
for i in range(77):
    scale[:,i] = scale[:,i]*domain[i]

In [ ]:
for i in range(24):
    scaled[:,i] = scale[:,i]

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
var= torch.FloatTensor(scale).to(device)

Split training, validation, and test sets

In [ ]:
def splitData(var,per_val,per_test):
    num_val=int(len(var)*per_val)
    num_test=int(len(var)*per_test)
    train_size=int(len(var)-num_val-num_test)
    train_data=var[0:train_size]
    val_data=var[train_size:train_size+num_val]
    test_data=var[train_size+num_val:train_size+num_val+num_test]
    return train_data,val_data,test_data

In [ ]:
train_data,val_data,test_data=splitData(var,0.1,0.1)

In [ ]:
input_size = train_data.shape[1]
print(input_size)

In [ ]:
print(train_data.shape)

In [ ]:
print('The length of train data, validation data and test data are:',len(train_data),',',len(val_data),',',len(test_data))

In [ ]:
from torch.utils.data import DataLoader, Dataset
class MyDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __getitem__(self, item):
        return self.data[item]

    def __len__(self):
        return len(self.data)

In [ ]:
batch_size = 8

In [ ]:
train_window = 8
def create_train_sequence(input_data, tw):
    inout_seq = []
    L = len(input_data)
    for i in range(L-tw):
        train_seq = input_data[i:i+tw]
        train_label = input_data[i+tw:i+tw+1,0]
        inout_seq.append((train_seq ,train_label))
    return inout_seq

In [ ]:
import torch.utils.data as data
train_inout_seq = create_train_sequence(train_data, train_window)
print('The total number of train windows is',len(train_inout_seq))
train_inout_seq = MyDataset(train_inout_seq)
train_inout_seq = DataLoader(dataset=train_inout_seq, batch_size=batch_size, shuffle=False, num_workers=0, drop_last=True)

In [ ]:
print(len(train_inout_seq))

In [ ]:
for (seq,label) in train_inout_seq:
    print(seq.shape)
    print(label)
    break

# Training preparation

In [ ]:
import copy

loss_train_dic={
    'BP':[],
    'RNN':[],
    'LSTM':[],
    'CNN':[],
    'BiLSTM':[],
    'CNNLSTM':[],
    'BiLSTMwithAttention':[],
    'CNNLSTMwithAttention':[],
    'LSTNet':[],
}
loss_val_dic=loss_train_dic
value_train_dic=loss_train_dic
value_val_dic=loss_train_dic

In [ ]:
def train_model(attr,model):
    model.train()
    best_model = model
    flag = 1
    path = './'+attr+'_aqi.pt'
    print('train',attr,'model')
    for i in range(epochs):
        model.train()
        #train
        add=0
        count=0
        for (seq,label) in train_inout_seq: 
            count+=1
            optimizer_aqi.zero_grad()
            label=label.to(device)
#             model.hidden_cell = (torch.zeros(2, 1, model.hidden_layer_size).to(device),torch.zeros(2, 1, model.hidden_layer_size).to(device))
            y_pred = model(seq)

            if(i==epochs-1):  
                value_train_dic[attr].append(y_pred)
            
            single_loss = loss_function(y_pred, label)   
            
            add+=single_loss
            single_loss.backward()
            optimizer_aqi.step()
        print("the learning rate of %d epoch：%f" % (i, optimizer_aqi.param_groups[0]['lr']))
        scheduler.step()
        loss_train=add/count
        loss_train_dic[attr].append(loss_train)
        
        #val
        add1=0 
        
        len_val = len(val_data)
        model.eval()
        
        val_inputs=train_data[-train_window:,:]
        val_list=[]
        
        model.eval()
    #     fut_pred = 650
        for n in range(len_val):
            seq = val_inputs[-train_window:].to(device)
            with torch.no_grad():
                model.hidden = (torch.zeros(1, 1, model.hidden_layer_size),
                              torch.zeros(1, 1, model.hidden_layer_size))
                seq = seq.unsqueeze(0)
#               ConvLSTM
#                 seq = seq.unsqueeze(0)
#                 seq = seq.unsqueeze(2)
                y_pred=model(seq)
                # Convlstm
#                 y_pred = torch.tensor(y_pred).view(-1,1)
#                 y_pred = y_pred.to(device)
                # Convlstm
#                 val_list.append(y_pred)
            
                loss = loss_function(y_pred,val_data[n,0].view(1,-1))
                add1 += loss
                temp=copy.deepcopy(val_data[n]).view(1,-1)

                val_inputs=torch.cat((val_inputs,temp),0)
                val_list.append(np.array(y_pred.cpu())[0])
        
#         plt.plot(np.array(val_data.cpu())[:,0],color="red",label='real value')
#         plt.plot(val_list,label='prediction')
#         plt.show()
        
        loss_val=add1/len_val
        loss_val_dic[attr].append(loss_val)
        if loss_val<flag:
            flag = loss_val
            best_model = model
            torch.save(best_model.state_dict(), path)
        print(f'epoch: {i:3}  train_loss:{loss_train:10.8f} val_loss:{loss_val:10.8f}')
    model = best_model
    print('----------------------')
    return best_model

In [ ]:
from math import sqrt
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error

def test_model(attr,model):
    temp=torch.cat((train_data,val_data))
    print(temp.shape)
    test_inputs=temp[-train_window:,:]
    print(test_inputs.shape)

    fut_pred = len(test_data)
    test_list=[]
    test_results=copy.deepcopy(test_data)

    model.eval()
    for i in range(fut_pred):
        seq = test_inputs[-train_window:].to(device)
        with torch.no_grad():
#             model.hidden = (torch.zeros(1, 1, model.hidden_layer_size),
#                           torch.zeros(1, 1, model.hidden_layer_size))
            seq = seq.unsqueeze(0)
            y_pred=model(seq)
            
            temp=copy.deepcopy(test_data[i]).view(1,-1)
#             temp[0]=y_pred
            test_inputs=torch.cat((test_inputs,temp),0)
            test_results[i,0]=y_pred

    local_path = ''


    actual_predictions = scaler.inverse_transform(np.array(test_results.cpu()))



    fig, ax = plt.subplots()
    plt.rcParams['font.sans-serif'] = ['Times New Roman']
    plt.rcParams['figure.figsize'] = (10,5)
    x = np.arange(len(train_data)+len(val_data), len(dataset), 1)

    data_target = origin[:,0]
    plt.plot(data_target[len(dataset)-len(test_data):] ,'r',label='Test data')
    plt.plot(actual_predictions[:,0],label='Prediction')

#     plt.title('days vs '+attr)
    plt.legend(loc='upper right',fontsize=12)
    plt.xlabel('Time',fontsize=16)
    plt.ylabel('AQI',fontsize=16)
    plt.tick_params(labelsize=12)
    path = local_path + attr + '_aqi.png'
    plt.savefig(path,dpi=600)
    plt.show()

    y_true=data_target[len(dataset)-len(test_data):]
    y_pred=actual_predictions[:,0]

    print("mae:", mean_absolute_error(y_true, y_pred))
    print("mse:", mean_squared_error(y_true, y_pred))
    print("rmse:", sqrt(mean_squared_error(y_true, y_pred)))
    print("r2 score:", r2_score(y_true, y_pred))
    print("mape:", mean_absolute_percentage_error(y_true, y_pred))

    y_pred=pd.DataFrame(y_pred)

# BP

In [ ]:
# window = 30, epoch=400

from torch import nn
import torch.nn.init as init
import torch.nn.functional as F
class BP(nn.Module):
    def __init__(self,input_size=24,hidden_layer_size=100,output_size=1):
        super().__init__()
        self.hidden_layer_size=hidden_layer_size
        self.flatten = nn.Flatten()
        self.linear1=nn.Linear(input_size*train_window,hidden_layer_size)
        self.linear2=nn.Linear(hidden_layer_size,output_size)
        
    def forward(self,input_seq):
        input_seq = input_seq.permute(0,2,1)
        out = self.flatten(input_seq)
        out=self.linear1(out)
#         out=torch.tanh(out)
        predictions = self.linear2(out)
#         print(predictions)
        return predictions

from torch.optim.lr_scheduler import StepLR
device=torch.device('cuda')
model_aqi = BP().to(device)
loss_function=nn.MSELoss()
optimizer_aqi = torch.optim.Adam(model_aqi.parameters(), lr=0.0001)
scheduler = StepLR(optimizer_aqi, step_size=5, gamma=0.95)
epochs = 300

In [ ]:
# BP
best_model = train_model('BP',model_aqi)

In [ ]:
# BP
test_model('BP',model_aqi)

# RNN

In [ ]:
from torch import nn
import torch.nn.init as init
import torch.nn.functional as F
class RNN(nn.Module):
    def __init__(self,input_size=77,hidden_layer_size=256,output_size=1,num_layers=2):
        super().__init__()
        self.hidden_layer_size=hidden_layer_size
        self.rnn=nn.RNN(input_size,hidden_layer_size,num_layers)
        self.linear1=nn.Linear(hidden_layer_size,hidden_layer_size)
        self.linear2=nn.Linear(hidden_layer_size,output_size)
        self.dropout=nn.Dropout(0.2)
#         self.hidden_cell=(torch.zeros(num_layers,1,self.hidden_layer_size),torch.zeros(num_layers,1,self.hidden_layer_size))
        init_rnn(self.rnn,'xavier')
        
    def forward(self,input_seq):
        input_seq = input_seq.permute(1,0,2)
        rnn_out, self.hidden_cell = self.rnn(input_seq)
        rnn_out = self.dropout(rnn_out)
        out=self.linear1(rnn_out)
#         out=torch.tanh(out)
        predictions = self.linear2(out)
        return predictions[-1]


def init_rnn(x, type='uniform'):
    for layer in x._all_weights:
        for w in layer:
            if 'weight' in w:
                if type == 'xavier':
                    init.xavier_normal_(getattr(x, w))
                elif type == 'uniform':
                    stdv = 1.0 / math.sqrt(x.hidden_size)
                    init.uniform_(getattr(x, w), -stdv, stdv)
                elif type == 'normal':
                    stdv = 1.0 / math.sqrt(x.hidden_size)
                    init.normal_(getattr(x, w), .0, stdv)
                else:
                    raise ValueError
                    
from torch.optim.lr_scheduler import StepLR
device=torch.device('cuda')
model_aqi = RNN().to(device)
loss_function=nn.MSELoss()
optimizer_aqi = torch.optim.Adam(model_aqi.parameters(), lr=0.0001)
scheduler = StepLR(optimizer_aqi, step_size=5, gamma=0.95)
epochs = 200

In [ ]:
# RNN
best_model = train_model('RNN',model_aqi)

In [ ]:
# RNN
test_model('RNN',best_model)

# LSTM

In [ ]:
from torch import nn
import torch.nn.init as init
import torch.nn.functional as F
class LSTM(nn.Module):
    def __init__(self,input_size=77,hidden_layer_size=256,output_size=1,num_layers=2):
        super().__init__()
        self.hidden_layer_size=hidden_layer_size
        self.lstm=nn.LSTM(input_size,hidden_layer_size,num_layers)
        self.linear1=nn.Linear(hidden_layer_size,hidden_layer_size)
        self.linear2=nn.Linear(hidden_layer_size,output_size)
        self.dropout=nn.Dropout(0.2)
#         self.hidden_cell=(torch.zeros(num_layers,1,self.hidden_layer_size),torch.zeros(num_layers,1,self.hidden_layer_size))
        init_rnn(self.lstm,'xavier')
        
    def forward(self,input_seq):
        input_seq = input_seq.permute(1,0,2)
        lstm_out, self.hidden_cell = self.lstm(input_seq)
        lstm_out = self.dropout(lstm_out)
        out=self.linear1(lstm_out)
#         out=torch.tanh(out)
        predictions = self.linear2(out)
        return predictions[-1]


def init_rnn(x, type='uniform'):
    for layer in x._all_weights:
        for w in layer:
            if 'weight' in w:
                if type == 'xavier':
                    init.xavier_normal_(getattr(x, w))
                elif type == 'uniform':
                    stdv = 1.0 / math.sqrt(x.hidden_size)
                    init.uniform_(getattr(x, w), -stdv, stdv)
                elif type == 'normal':
                    stdv = 1.0 / math.sqrt(x.hidden_size)
                    init.normal_(getattr(x, w), .0, stdv)
                else:
                    raise ValueError
                    
from torch.optim.lr_scheduler import StepLR
device=torch.device('cuda')
model_aqi = LSTM().to(device)
loss_function=nn.MSELoss()
optimizer_aqi = torch.optim.Adam(model_aqi.parameters(), lr=0.0001)
scheduler = StepLR(optimizer_aqi, step_size=5, gamma=0.95)
epochs = 200

In [ ]:
# LSTM
best_model = train_model('LSTM',model_aqi)

In [ ]:
# LSTM
test_model('LSTM',best_model)

# CNN

In [ ]:
# window = 30

from torch import nn
import torch.nn.init as init
import torch.nn.functional as F
class CNN(nn.Module):
    def __init__(self,input_size=77,hidden_layer_size=100,output_size=1):
        super().__init__()
        self.hidden_layer_size=hidden_layer_size
        self.conv1d1=nn.Conv1d(in_channels=input_size,out_channels =16, kernel_size = 1)
        self.conv1d2=nn.Conv1d(in_channels=16,out_channels =32, kernel_size = 1)
        self.maxpool1d = nn.MaxPool1d(kernel_size=2)
        self.flatten = nn.Flatten()
        self.linear1=nn.Linear(16*train_window,hidden_layer_size)
        self.linear2=nn.Linear(hidden_layer_size,output_size)
        
    def forward(self,input_seq):
        input_seq = input_seq.permute(0,2,1)
        input_seq = self.conv1d1(input_seq)
        input_seq = self.conv1d2(input_seq)
        input_seq = self.maxpool1d(input_seq)
        out = self.flatten(input_seq)
        out=self.linear1(out)
#         out=torch.tanh(out)
        predictions = self.linear2(out)
#         print(predictions)
        return predictions

from torch.optim.lr_scheduler import StepLR
device=torch.device('cuda')
model_aqi = CNN().to(device)
loss_function=nn.MSELoss()
optimizer_aqi = torch.optim.Adam(model_aqi.parameters(), lr=0.0001)
scheduler = StepLR(optimizer_aqi, step_size=5, gamma=0.95)
epochs = 200

In [ ]:
# LSTM
best_model = train_model('CNN',model_aqi)

In [ ]:
# LSTM
test_model('CNN',best_model)

# CNN-LSTM

In [ ]:
#CNN-LSTM with attention
from torch import nn
import torch.nn.functional as F
import torch.nn.init as init
class CNNLSTM(nn.Module):
    def __init__(self,input_size=20,hidden_layer_size=256,output_size=1,num_layers=2):
        super().__init__()
        self.input_size=input_size
        self.output_size=output_size
        self.hidden_layer_size=hidden_layer_size
        self.num_layers=num_layers

        self.conv1d1=nn.Conv1d(in_channels=input_size,out_channels =32, kernel_size = 1)
        self.conv1d2=nn.Conv1d(in_channels=32,out_channels =64, kernel_size = 1)
        self.activation=nn.ELU()
        self.maxpool1d = nn.MaxPool1d(kernel_size=2)
#         self.lstm1=nn.LSTM(32,hidden_layer_size1,num_layers)
        self.lstm=nn.LSTM(64,hidden_layer_size,num_layers,dropout=0.2)
        
        self.linear1=nn.Linear(self.hidden_layer_size,self.hidden_layer_size)
        self.linear2=nn.Linear(self.hidden_layer_size,output_size)
        init_rnn(self.lstm,'xavier')
#         init_rnn(self.lstm2,'xavier')
        self.dropout=nn.Dropout(0.2)
        self.attn_weights=None


    def attention_net(self, lstm_output, final_state):

        lstm_output = lstm_output.permute(1, 0, 2)
#         lstm_output [1, 59 ,50]
#         final_state torch.Size([2, 1, 50])
#         print(final_state.shape)
        hidden = final_state.view(-1,self.hidden_layer_size, 1)   # hidden : [batch_size, n_hidden * num_directions, 1(=n_layer)]
#         torch.Size([1, 50, 2])
        self.attn_weights = torch.bmm(lstm_output, hidden).squeeze(2) # attn_weights : [batch_size, n_step]
#         print(self.attn_weights.shape)
        soft_attn_weights = torch.tanh(self.attn_weights)
#         print(soft_attn_weights.shape)
        context = torch.bmm(lstm_output.transpose(1, 2), soft_attn_weights.unsqueeze(2)).squeeze(2)
        return context # context : [batch_size, n_hidden * num_directions]
    
    def forward(self,input_seq):
        batch_size = input_seq.shape[0]
        input_seq=input_seq.permute(0,2,1)
        input_seq=self.conv1d1(input_seq)
        input_seq=self.activation(input_seq)
        input_seq=self.conv1d2(input_seq)
        input_seq=self.activation(input_seq)
        input_seq=input_seq.permute(2,0,1)
        lstm_out, self.hidden_cell = self.lstm(input_seq)
        lstm_out = self.dropout(lstm_out)
        # torch.Size([59, 1, 50])
        # torch.Size([1, 50, 2])
#         print(lstm_out.shape)
        out=self.linear1(lstm_out)
        # torch.Size([2, 50])
#         print(out.shape)
        predictions = self.linear2(out)
#         print(predictions.shape)
        # torch.Size([2, 1])
#         predictions = predictions[:, -1, :]
        return predictions[-1]

def init_rnn(x, type='uniform'):
    for layer in x._all_weights:
        for w in layer:
            if 'weight' in w:
                if type == 'xavier':
                    init.xavier_normal_(getattr(x, w))
                elif type == 'uniform':
                    stdv = 1.0 / math.sqrt(x.hidden_size)
                    init.uniform_(getattr(x, w), -stdv, stdv)
                elif type == 'normal':
                    stdv = 1.0 / math.sqrt(x.hidden_size)
                    init.normal_(getattr(x, w), .0, stdv)
                else:
                    raise ValueError

from torch.optim.lr_scheduler import StepLR
device=torch.device('cuda')
model_aqi = CNNLSTM().to(device)
loss_function=nn.MSELoss()
optimizer_aqi = torch.optim.Adam(model_aqi.parameters(), lr=0.0001)
scheduler = StepLR(optimizer_aqi, step_size=5, gamma=0.95)
epochs = 200

In [ ]:
# CNNLSTM
best_model = train_model('CNNLSTM',model_aqi)

In [ ]:
# CNNLSTM
test_model('CNNLSTM',best_model)

In [ ]:
# LSTNet
from torch import nn
import torch.nn.functional as F
import torch.nn.init as init
class LSTNet(nn.Module):
    def __init__(self,input_size=77,hidden_layer_size=256,output_size=1,num_layers=2,window=8):
        super().__init__()
        self.input_size=input_size
        self.output_size=output_size
        self.hidden_layer_size=hidden_layer_size
        self.num_layers=num_layers
        self.hidR = 32 
        self.hidC = 32 
        self.hidS = 3 
        self.kernel_size = 1

        self.conv1d1=nn.Conv1d(in_channels=input_size,out_channels =self.hidC, kernel_size = self.kernel_size)
        self.conv1d2=nn.Conv1d(in_channels=self.hidC,out_channels =self.hidR, kernel_size = self.kernel_size)
        self.activation=nn.ELU()
        self.maxpool1d = nn.MaxPool1d(kernel_size=2)
#         self.lstm1=nn.LSTM(32,hidden_layer_size1,num_layers)
        self.GRU1=nn.GRU(self.hidR,self.hidden_layer_size,num_layers,dropout=0.2)
        init_rnn(self.GRU1,'xavier')
#         init_rnn(self.lstm2,'xavier')
        self.dropout=nn.Dropout(0.2)
        self.attn_weights=None
        self.skip = 4 
        self.pt = int((window - self.kernel_size)/self.skip) 
        self.hw = 3 
        if (self.skip > 0):
            self.GRUskip = nn.GRU(self.hidC, self.hidS)
            self.linear1 = nn.Linear(self.hidden_layer_size + self.skip * self.hidS, self.output_size)
        else:
            self.linear1 = nn.Linear(self.hidden_layer_size, self.output_size)
        if (self.hw > 0):
            self.highway = nn.Linear(self.hw, self.output_size)
        self.linear2 = nn.Linear(input_size, self.output_size)
        
        
    
    def forward(self,input_seq):
        batch_size = input_seq.shape[0]
        c=input_seq.permute(0,2,1)
        c=self.conv1d1(c)
        c=self.activation(c)
        c=self.conv1d2(c)
        c=self.activation(c)
        r=c.permute(2,0,1)
        GRU_out, self.hidden_cell = self.GRU1(r)
        GRU_out = GRU_out[-1]
        GRU_out = self.dropout(GRU_out)

        #skip-rnn
        if (self.skip > 0):
            s = c[:,:, int(-self.pt * self.skip):].contiguous()
            s = s.view(batch_size, self.hidC, self.pt, self.skip)
            s = s.permute(2,0,3,1).contiguous()
            s = s.view(self.pt, batch_size * self.skip, self.hidC)
            _, s = self.GRUskip(s)
            s = s.view(batch_size, self.skip * self.hidS)
            s = self.dropout(s)
            GRU_out = torch.cat((GRU_out,s),1)
        
        res = self.linear1(GRU_out)
        
        #highway
        if (self.hw > 0):
            z = input_seq[:, -self.hw:, :]
            z = z.permute(0,2,1)
            z = self.highway(z)
            z = z.permute(1,0,2)
            z = z[-1]
            predictions = res + z
            
        return predictions

def init_rnn(x, type='uniform'):
    for layer in x._all_weights:
        for w in layer:
            if 'weight' in w:
                if type == 'xavier':
                    init.xavier_normal_(getattr(x, w))
                elif type == 'uniform':
                    stdv = 1.0 / math.sqrt(x.hidden_size)
                    init.uniform_(getattr(x, w), -stdv, stdv)
                elif type == 'normal':
                    stdv = 1.0 / math.sqrt(x.hidden_size)
                    init.normal_(getattr(x, w), .0, stdv)
                else:
                    raise ValueError

from torch.optim.lr_scheduler import StepLR
device=torch.device('cuda')
model_aqi = LSTNet().to(device)
loss_function=nn.MSELoss()
optimizer_aqi = torch.optim.Adam(model_aqi.parameters(), lr=0.0001)
scheduler = StepLR(optimizer_aqi, step_size=5, gamma=0.95)
epochs = 200

In [ ]:
# LSTNet
best_model = train_model('LSTNet', model_aqi)

In [ ]:
# LSTNet
test_model('LSTNet',best_model)

# CNN-LSTM with attention

In [ ]:
#CNN-LSTM with attention
from torch import nn
import torch.nn.functional as F
import torch.nn.init as init
class CNNLSTMwithAttention(nn.Module):
    def __init__(self,input_size=24,hidden_layer_size=256,output_size=1,num_layers=2):
        super().__init__()
        self.input_size=input_size
        self.output_size=output_size
        self.hidden_layer_size=hidden_layer_size
        self.num_layers=num_layers

        self.conv1d1=nn.Conv1d(in_channels=input_size,out_channels =32, kernel_size = 1)
        self.conv1d2=nn.Conv1d(in_channels=32,out_channels =64, kernel_size = 1)
        self.maxpool1d = nn.MaxPool1d(kernel_size=2)
#         self.lstm1=nn.LSTM(32,hidden_layer_size1,num_layers)
        self.lstm=nn.LSTM(64,hidden_layer_size,num_layers,dropout=0.2)
        
        self.linear1=nn.Linear(self.hidden_layer_size,self.hidden_layer_size)
        self.linear2=nn.Linear(self.hidden_layer_size,output_size)
        init_rnn(self.lstm,'xavier')
#         init_rnn(self.lstm2,'xavier')
        self.dropout=nn.Dropout(0.2)
        self.attn_weights=None


    def attention_net(self, lstm_output, final_state):

        lstm_output = lstm_output.permute(1, 0, 2)
#         lstm_output [1, 59 ,50]
#         final_state torch.Size([2, 1, 50])
#         print(final_state.shape)
        hidden = final_state.view(-1,self.hidden_layer_size, 1)   # hidden : [batch_size, n_hidden * num_directions, 1(=n_layer)]
#         torch.Size([1, 50, 2])
        self.attn_weights = torch.bmm(lstm_output, hidden).squeeze(2) # attn_weights : [batch_size, n_step]
#         print(self.attn_weights.shape)
        soft_attn_weights = torch.tanh(self.attn_weights)
#         print(soft_attn_weights.shape)
        context = torch.bmm(lstm_output.transpose(1, 2), soft_attn_weights.unsqueeze(2)).squeeze(2)
        return context # context : [batch_size, n_hidden * num_directions]
    
    def forward(self,input_seq):
#         print(input_seq.shape)
        batch_size = input_seq.shape[0]
        input_seq=input_seq.permute(0,2,1)
        # torch.Size([1, 6, 60])
        input_seq=self.conv1d1(input_seq)
#         torch.Size([1, 100, 29])
        input_seq=self.conv1d2(input_seq)
#         torch.Size([1, 50, 28])
#         input_seq=self.maxpool1d(input_seq)
#         torch.Size([1, 50, 14])
        input_seq=input_seq.permute(2,0,1)
        # torch.Size([1, 59, 20])
#         input_seq=input_seq.reshape(len(input_seq[0]),batch_size,input_seq.shape[2])
        # torch.Size([59, 1, 20])
        lstm_out, self.hidden_cell = self.lstm(input_seq)
        lstm_out = self.dropout(lstm_out)
        # torch.Size([59, 1, 50])
        attn_output = self.attention_net(lstm_out,self.hidden_cell[0][1].unsqueeze(0))
        # torch.Size([1, 50, 2])
        out=self.linear1(attn_output.view(-1,self.hidden_layer_size))
        # torch.Size([2, 50])
        predictions = self.linear2(out)
        # torch.Size([2, 1])
#         predictions = predictions[:, -1, :]
        return predictions

def init_rnn(x, type='uniform'):
    for layer in x._all_weights:
        for w in layer:
            if 'weight' in w:
                if type == 'xavier':
                    init.xavier_normal_(getattr(x, w))
                elif type == 'uniform':
                    stdv = 1.0 / math.sqrt(x.hidden_size)
                    init.uniform_(getattr(x, w), -stdv, stdv)
                elif type == 'normal':
                    stdv = 1.0 / math.sqrt(x.hidden_size)
                    init.normal_(getattr(x, w), .0, stdv)
                else:
                    raise ValueError

from torch.optim.lr_scheduler import StepLR
device=torch.device('cuda')
model_aqi = CNNLSTMwithAttention().to(device)
loss_function=nn.MSELoss()
optimizer_aqi = torch.optim.Adam(model_aqi.parameters(), lr=0.0001)
scheduler = StepLR(optimizer_aqi, step_size=5, gamma=0.95)
epochs = 200

In [ ]:
# CNNLSTMwithAttention
best_model = train_model('CNNLSTMwithAttention',model_aqi)

In [ ]:
# CNNLSTMwithAttention
test_model('CNNLSTMwithAttention',best_model)

# BiLSTM

In [ ]:
#Bilstm with attention
from torch import nn
import torch.nn.init as init
import torch.nn.functional as F
class BiLSTM(nn.Module):
    def __init__(self,input_size=77,hidden_layer_size=256,output_size=1,num_layers=1):
        super().__init__()
        self.hidden_layer_size=hidden_layer_size
        self.bilstm=nn.LSTM(input_size,hidden_layer_size,num_layers,bidirectional=True)
        self.linear1=nn.Linear(hidden_layer_size*2,hidden_layer_size)
        self.linear2=nn.Linear(hidden_layer_size,output_size)
#         self.hidden_cell=(torch.zeros(num_layers*2,8,self.hidden_layer_size).to(device),torch.zeros(num_layers*2,8,self.hidden_layer_size).to(device))
        self.num_layers=num_layers
        self.dropout=nn.Dropout(0.2)
        init_rnn(self.bilstm,'xavier')

    def attention_net(self, lstm_output, final_state):
        lstm_output = lstm_output.permute(1, 0, 2)
        batch_size = len(lstm_output)
        hidden = final_state.view(batch_size, -1, 1)  
        # hidden : [batch_size, n_hidden * num_directions(=2), n_layer(=1)]
        attn_weights = torch.bmm(lstm_output, hidden).squeeze(2) 
        # attn_weights : [batch_size, n_step]
        soft_attn_weights = F.softmax(attn_weights, 1)
        # context : [batch_size, n_hidden * num_directions(=2)]
        context = torch.bmm(lstm_output.transpose(1, 2), soft_attn_weights.unsqueeze(2)).squeeze(2)
        context = self.dropout(context)
        return context

    def forward(self,input_seq):
        input_seq = input_seq.permute(1,0,2)
        bilstm_out, self.hidden_cell = self.bilstm(input_seq)
#         attn_output = self.attention_net(bilstm_out,self.hidden_cell[0])
        # torch.Size([1, 50, 2])
        bilstml_out=self.dropout(bilstm_out)
        out=self.linear1(bilstm_out)
        # torch.Size([2, 50])
#         out=self.linear1(out.view(len(input_seq)*2, -1))
#         out=torch.tanh(out)
        predictions = self.linear2(out)
        return predictions[-1]


def init_rnn(x, type='uniform'):
    for layer in x._all_weights:
        for w in layer:
            if 'weight' in w:
                if type == 'xavier':
                    init.xavier_normal_(getattr(x, w))
                elif type == 'uniform':
                    stdv = 1.0 / math.sqrt(x.hidden_size)
                    init.uniform_(getattr(x, w), -stdv, stdv)
                elif type == 'normal':
                    stdv = 1.0 / math.sqrt(x.hidden_size)
                    init.normal_(getattr(x, w), .0, stdv)
                else:
                    raise ValueError

from torch.optim.lr_scheduler import StepLR
device=torch.device('cuda')
model_aqi = BiLSTM().to(device)
loss_function=nn.MSELoss()
optimizer_aqi = torch.optim.Adam(model_aqi.parameters(), lr=0.0001)
scheduler = StepLR(optimizer_aqi, step_size=5, gamma=0.95)
epochs = 200

In [ ]:
# BiLSTM
best_model = train_model('BiLSTM',model_aqi)

In [ ]:
# BiLSTM
test_model('BiLSTM',best_model)

# BiLSTM with attention

In [ ]:
#Bilstm with attention
from torch import nn
import torch.nn.init as init
import torch.nn.functional as F
class BiLSTMwithAttention(nn.Module):
    def __init__(self,input_size=24,hidden_layer_size=256,output_size=1,num_layers=1):
        super().__init__()
        self.hidden_layer_size=hidden_layer_size
        self.bilstm=nn.LSTM(input_size,hidden_layer_size,num_layers,bidirectional=True)
        self.linear1=nn.Linear(hidden_layer_size*2,hidden_layer_size)
        self.linear2=nn.Linear(hidden_layer_size,output_size)
#         self.hidden_cell=(torch.zeros(num_layers*2,8,self.hidden_layer_size).to(device),torch.zeros(num_layers*2,8,self.hidden_layer_size).to(device))
        self.num_layers=num_layers
        self.dropout=nn.Dropout(0.2)
        init_rnn(self.bilstm,'xavier')

    def attention_net(self, lstm_output, final_state):
        lstm_output = lstm_output.permute(1, 0, 2)
        batch_size = len(lstm_output)
        hidden = final_state.view(batch_size, -1, 1)  
        # hidden : [batch_size, n_hidden * num_directions(=2), n_layer(=1)]
        attn_weights = torch.bmm(lstm_output, hidden).squeeze(2) 
        # attn_weights : [batch_size, n_step]
        soft_attn_weights = F.softmax(attn_weights, 1)
        # context : [batch_size, n_hidden * num_directions(=2)]
        context = torch.bmm(lstm_output.transpose(1, 2), soft_attn_weights.unsqueeze(2)).squeeze(2)
        context = self.dropout(context)
        return context
    
    def forward(self,input_seq):
        input_seq = input_seq.permute(1,0,2)
        bilstm_out, self.hidden_cell = self.bilstm(input_seq)
        attn_output = self.attention_net(bilstm_out,self.hidden_cell[0])
        # torch.Size([1, 50, 2])
        out=self.linear1(attn_output.view(-1,self.hidden_layer_size*2))
#         out=self.dropout(out)
        # torch.Size([2, 50])
#         out=self.linear1(out.view(len(input_seq)*2, -1))
#         out=torch.tanh(out)
        predictions = self.linear2(out)
        return predictions


def init_rnn(x, type='uniform'):
    for layer in x._all_weights:
        for w in layer:
            if 'weight' in w:
                if type == 'xavier':
                    init.xavier_normal_(getattr(x, w))
                elif type == 'uniform':
                    stdv = 1.0 / math.sqrt(x.hidden_size)
                    init.uniform_(getattr(x, w), -stdv, stdv)
                elif type == 'normal':
                    stdv = 1.0 / math.sqrt(x.hidden_size)
                    init.normal_(getattr(x, w), .0, stdv)
                else:
                    raise ValueError

from torch.optim.lr_scheduler import StepLR
device=torch.device('cuda')
model_aqi = BiLSTMwithAttention().to(device)
loss_function=nn.MSELoss()
optimizer_aqi = torch.optim.Adam(model_aqi.parameters(), lr=0.0001)
scheduler = StepLR(optimizer_aqi, step_size=5, gamma=0.95)
epochs = 200

In [ ]:
# BiLSTM
best_model = train_model('BiLSTMwithAttention',model_aqi)

In [ ]:
# BiLSTM
test_model('BiLSTMwithAttention',best_model)

In [ ]:
import matplotlib.pyplot as plt
plt.get_cmap('Set3')

In [ ]:
your_path = ''
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.rcParams["font.sans-serif"]=['Times New Roman']
plt.rcParams["axes.unicode_minus"]=False
column = ['pm2.5','pm10','SO2','NO2','CO','O3']
d_weight = [0.8702,0.6893,0.7150,0.7416,0.6655,0.7929]
select = [0,2,3,4,5,6]
colormap = plt.get_cmap('Set3')(select)


for i in range(len(column)):
    plt.bar(column[i],d_weight[i],width=0.5, color=colormap[i])
for a,b in zip(column,d_weight): 
    plt.text(a,b,'%.2f'%b,ha='center',va='bottom',fontsize=16);

plt.tick_params(labelsize=16)
    
plt.title("domain weight", fontsize=16)
plt.savefig(your_path, dpi=600)
plt.show()